In [28]:
pip install chromadb

Note: you may need to restart the kernel to use updated packages.


In [1]:
import chromadb

client = chromadb.CloudClient(
  api_key='ck-GDUbzXXARSWT1JoFwsZNyF51Bp89RBzkVdzVmx3kXTbc',
  tenant='e313d9a7-5504-4844-8965-506a1c58e390',
  database='ChromaDB_image_text'
)

In [2]:
client

In [31]:
pip install pymupdf sentence-transformers chromadb pillow

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
from typing import List,Dict, Any

import fitz
from PIL  import Image
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb

c:\Users\SHREE\anaconda3\envs\genai\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
PDF_PATH = r"sample_data/attention-is-all-you-need.pdf"
FIGURES_DIR = "figures"
os.makedirs(FIGURES_DIR,exist_ok=True)

In [5]:
import chromadb

client = chromadb.CloudClient(
  api_key='ck-GDUbzXXARSWT1JoFwsZNyF51Bp89RBzkVdzVmx3kXTbc',
  tenant='e313d9a7-5504-4844-8965-506a1c58e390',
  database='ChromaDB_image_text'
)

In [6]:
FIG_COLLECTION_NAME  = "attention_figures"
TEXT_COLLECTION_NAME = "attention_text_chunks"

### we use have multiple model with accuracy 
1) clip-vit-b-32 model this give me 60% accuracy
2) BAAI/EVA-CLIP-18B this is a give me 80% accuracy multi model
3) blip-2 series  this is also best models

In [7]:
clip_model = SentenceTransformer("clip-ViT-B-32")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [8]:
def embed_image(path: str) -> np.ndarray:
    img = Image.open(path).convert("RGB")
    emb = clip_model.encode([img], convert_to_numpy = True, show_progress_bar = False)
    return emb[0]

def embed_text(text: str) -> np.ndarray:
    emb = clip_model.encode([text], convert_to_numpy = True, show_progress_bar = False)
    return emb[0]

In [9]:
doc = fitz.open(PDF_PATH)

In [10]:
doc

Document('sample_data/attention-is-all-you-need.pdf')

In [11]:
for i in doc:
    print(i.get_text("blocks"))

[(211.48800659179688, 99.7807846069336, 399.8930358886719, 116.99618530273438, 'Attention Is All You Need\n', 0, 0), (116.68070983886719, 185.17320251464844, 216.05758666992188, 217.97817993164062, 'Ashish Vaswani∗\nGoogle Brain\navaswani@google.com\n', 1, 0), (230.69180297851562, 185.17332458496094, 309.14739990234375, 217.97830200195312, 'Noam Shazeer∗\nGoogle Brain\nnoam@google.com\n', 2, 0), (323.78668212890625, 185.17344665527344, 407.47271728515625, 217.97842407226562, 'Niki Parmar∗\nGoogle Research\nnikip@google.com\n', 3, 0), (422.11187744140625, 185.17356872558594, 497.21026611328125, 217.97854614257812, 'Jakob Uszkoreit∗\nGoogle Research\nusz@google.com\n', 4, 0), (126.88175201416016, 235.1716766357422, 210.5675506591797, 267.9754333496094, 'Llion Jones∗\nGoogle Research\nllion@google.com\n', 5, 0), (235.40676879882812, 235.1716766357422, 340.0142517089844, 267.9754333496094, 'Aidan N. Gomez∗†\nUniversity of Toronto\naidan@cs.toronto.edu\n', 6, 0), (364.8485412597656, 235.171

In [12]:
for i in doc:
    print(i.get_images("blocks"))

[]
[]
[(94, 95, 455, 671, 8, 'DeviceRGB', '', 'R72', 'FlateDecode', 0), (95, 0, 455, 671, 8, 'DeviceGray', '', 'R71', 'FlateDecode', 0)]
[(103, 104, 250, 384, 8, 'DeviceRGB', '', 'R86', 'FlateDecode', 0), (104, 0, 250, 384, 8, 'DeviceGray', '', 'R85', 'FlateDecode', 0), (105, 106, 133, 265, 8, 'DeviceRGB', '', 'R84', 'FlateDecode', 0), (106, 0, 133, 265, 8, 'DeviceGray', '', 'R83', 'FlateDecode', 0)]
[]
[]
[]
[]
[]
[]
[]


In [13]:
doc.metadata

{'format': 'PDF 1.3',
 'title': 'Attention is All you Need',
 'author': 'Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin',
 'subject': 'The dominant sequence transduction models are based on complex recurrent orconvolutional neural networks in an encoder and decoder configuration. The best performing such models also connect the encoder and decoder through an attentionm echanisms. We propose a novel, simple network architecture based solely onan attention mechanism, dispensing with recurrence and convolutions entirely.Experiments on two machine translation tasks show these models to be superiorin quality while being more parallelizable and requiring significantly less timeto train. Our single model with 165 million parameters, achieves 27.5 BLEU onEnglish-to-German translation, improving over the existing best ensemble result by over 1 BLEU. On English-to-French translation, we outperform the previoussingle state-

In [14]:
image_records: List[Dict[str,Any]] =[]

print("Extracting Images and captions......")

for page_index, page in enumerate(doc,start = 1):
    blocks  = page.get_text("blocks")   # (x0 ,y0,x1, y1, text, .....)
    text_blocks = [b for b in blocks if b[4].strip()]  # keep non-empty text block
    
    images = page.get_images(full=True)
    print(f"page {page_index}: found {len(images)} images")
    
    for img_index , img in enumerate(images,start = 1):
        xref = img[0]
        
        # where is the image on the page  ? 
        rects = page.get_image_rects(xref)
        if not rects:
            continue
        rect = rects[0]
        
        below_blocks = [b for b in text_blocks if b[1] >= rect.y1]
        candidates = below_blocks if below_blocks else text_blocks
        
        if candidates:
             closest_block = min(candidates, key=lambda b : abs(b[1]  - rect.y1))
             caption_text = closest_block[4].strip().replace("\n"," ")
             
        else:
            caption_text=""
        
        # extract image binary and save to file
        img_data = doc.extract_image(xref)
        img_bytes = img_data["image"]
        img_ext = img_data.get("ext","png")
        
        img_filename = f"page_{page_index:02d}_img_{img_index}.{img_ext}"
        img_path = os.path.join(FIGURES_DIR,img_filename)
        
        with open(img_path,"wb") as f:
            f.write(img_bytes)
            
        image_records.append({
            "id":f"p{page_index}_img{img_index}",
            "page_number":page_index,
            "bbox_x0":float(rect.x0),
            "bbox_y0":float(rect.y0),
            "bbox_x1":float(rect.x1),
            "bbox_y1":float(rect.y1),
            "caption":caption_text,
            "image_path":img_path           
        })
        
print(f"Total extracted images: {len(image_records)}")

Extracting Images and captions......
page 1: found 0 images
page 2: found 0 images
page 3: found 2 images
page 4: found 4 images
page 5: found 0 images
page 6: found 0 images
page 7: found 0 images
page 8: found 0 images
page 9: found 0 images
page 10: found 0 images
page 11: found 0 images
Total extracted images: 3


In [15]:
text_records: List[Dict[str,Any]] = []
print("Extracting text chunks....")
MIN_CHARS = 80

for page_index, page in enumerate(doc,start = 1):
    blocks = page.get_text("blocks")
    for block_idx, b in enumerate(blocks):
        text = b[4].strip()
        if len(text) < MIN_CHARS:
            continue
        
        x0, y0, x1, y1 = b[0] , b[1], b[2], b[3]
        
        text_records.append({
            "id": f"p{page_index}_block{block_idx}",
            "page_number": page_index,
            "bbox_x0" : float(x0),
            "bbox_y0": float(y0),
            "bbox_x1": float(x1),
            "bbox_y1":float(y1),
            "text":text,
        })
print(f"Total extracted text chunks : {len(text_records)}")

Extracting text chunks....
Total extracted text chunks : 101


In [16]:
text_records

[{'id': 'p1_block10',
  'page_number': 1,
  'bbox_x0': 143.86599731445312,
  'bbox_y0': 360.1273193359375,
  'bbox_x1': 469.7874755859375,
  'bbox_y1': 501.0741882324219,
  'text': 'The dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing with recurrence and convolutions\nentirely. Experiments on two machine translation tasks show these models to\nbe superior in quality while being more parallelizable and requiring signiﬁcantly\nless time to train. Our model achieves 28.4 BLEU on the WMT 2014 English-\nto-German translation task, improving over the existing best results, including\nensembles, by over 2 BLEU. On the WMT 2014 English-to-French translation task,\nour model establishes a new sin

In [17]:
fig_collection = client.get_or_create_collection(
    name = FIG_COLLECTION_NAME,
    metadata = {"type":"text","source": "attension_pdf"}
)

text_collection = client.get_or_create_collection(
    name = TEXT_COLLECTION_NAME,
    metadata = {"type":"text","source": "attension_pdf"}
)

In [18]:
fig_ids = []
fig_embeddings = []
fig_metadatas = []
fig_documents = []
for rec in image_records:
    img_emb  = embed_image(rec["image_path"])
    
    fig_ids.append(rec["id"])
    fig_embeddings.append(img_emb.tolist())
    fig_metadatas.append({
        "page_number":int(rec["page_number"]),
        "bbox_x0":rec["bbox_x0"],
        "bbox_y0":rec["bbox_y0"],
        "bbox_x1":rec["bbox_x1"],
        "bbox_y1":rec["bbox_y1"],
        "image_path":rec["image_path"],
        "caption": rec["caption"],
    })
    fig_documents.append(
        f"Image from 'attention is all you need' on page {rec['page_number']}."
        f"caption: {rec['caption']}"
    )

if fig_ids:
    fig_collection.add(
        ids = fig_ids,
        embeddings=fig_embeddings,
        metadatas = fig_metadatas,
        documents = fig_documents,
    )
    


In [19]:
text_ids = []
text_embeddings = []
text_metadatas = []
text_documents = []

for rec in text_records:

    text = rec.get("text", "").strip()
    if not text:
        continue  # skip empty text blocks

    text_emb = embed_text(text)

    # ensure embedding is list
    if hasattr(text_emb, "tolist"):
        text_emb = text_emb.tolist()

    text_ids.append(rec["id"])
    text_embeddings.append(text_emb)

    text_metadatas.append({
        "page_number": int(rec["page_number"]),
        "bbox_x0": float(rec["bbox_x0"]),
        "bbox_y0": float(rec["bbox_y0"]),
        "bbox_x1": float(rec["bbox_x1"]),
        "bbox_y1": float(rec["bbox_y1"]),
    })

    # documents must be TEXT
    text_documents.append(text)

if text_ids:
    text_collection.add(
        ids=text_ids,
        embeddings=text_embeddings,
        metadatas=text_metadatas,
        documents=text_documents,
    )


In [20]:
from typing import List, Dict, Any

def answer_query(
    query: str,
    top_k_text: int = 5,
    top_k_img: int = 5,
    top_k_overall: int = 8,
) -> List[Dict[str, Any]]:
    """
    Search both text chunks and images in Chroma,
    merge results by distance, and return top hits.
    """

    q_emb = embed_text(query)
    combined: List[Dict[str, Any]] = []

    # -------- TEXT SEARCH --------
    text_results = text_collection.query(
        query_embeddings=[q_emb.tolist()],
        n_results=top_k_text,
        include=["metadatas", "documents", "distances"],
    )

    for meta, doc, dist in zip(
        text_results["metadatas"][0],
        text_results["documents"][0],
        text_results["distances"][0],
    ):
        combined.append({
            "type": "text",
            "distance": float(dist),
            "page_number": meta.get("page_number"),
            "content": doc,
        })

    # -------- IMAGE SEARCH --------
    fig_results = fig_collection.query(
        query_embeddings=[q_emb.tolist()],
        n_results=top_k_img,
        include=["metadatas", "documents", "distances"],
    )

    for meta, doc, dist in zip(
        fig_results["metadatas"][0],
        fig_results["documents"][0],
        fig_results["distances"][0],
    ):
        combined.append({
            "type": "image",
            "distance": float(dist),
            "page_number": meta.get("page_number"),
            "image_path": meta.get("image_path"),
            "caption": meta.get("caption", ""),
            "content": doc,
        })

    # -------- SORT & TRIM --------
    combined.sort(key=lambda x: x["distance"])
    combined = combined[:top_k_overall]

    # -------- PRINT RESULTS --------
    print(f"\n=== Query: {query} ===")
    for rank, item in enumerate(combined, start=1):
        print(f"\nRank {rank} | type: {item['type']} | distance: {item['distance']:.4f}")
        print("page:", item["page_number"])

        if item["type"] == "text":
            preview = item["content"].replace("\n", " ")
            print("text:", preview[:260] + "..." if len(preview) > 260 else preview)
        else:
            print("image path:", item["image_path"])
            print("caption:", item["caption"])

    return combined


In [21]:
answer_query("What is the Transformer architecture?",top_k_text=2,top_k_img=1,top_k_overall=3)


=== Query: What is the Transformer architecture? ===

Rank 1 | type: text | distance: 26.6708
page: 2
text: To the best of our knowledge, however, the Transformer is the ﬁrst transduction model relying entirely on self-attention to compute representations of its input and output without using sequence- aligned RNNs or convolution. In the following sections, we will ...

Rank 2 | type: text | distance: 32.1169
page: 9
text: For translation tasks, the Transformer can be trained signiﬁcantly faster than architectures based on recurrent or convolutional layers. On both WMT 2014 English-to-German and WMT 2014 English-to-French translation tasks, we achieve a new state of the art. In ...

Rank 3 | type: image | distance: 143.9729
page: 4
image path: figures/page_04_img_1.png
caption: Figure 2: (left) Scaled Dot-Product Attention. (right) Multi-Head Attention consists of several
attention layers running in parallel.


[{'type': 'text',
  'distance': 26.670824,
  'page_number': 2,
  'content': 'To the best of our knowledge, however, the Transformer is the ﬁrst transduction model relying\nentirely on self-attention to compute representations of its input and output without using sequence-\naligned RNNs or convolution. In the following sections, we will describe the Transformer, motivate\nself-attention and discuss its advantages over models such as [14, 15] and [8].'},
 {'type': 'text',
  'distance': 32.1169,
  'page_number': 9,
  'content': 'For translation tasks, the Transformer can be trained signiﬁcantly faster than architectures based\non recurrent or convolutional layers. On both WMT 2014 English-to-German and WMT 2014\nEnglish-to-French translation tasks, we achieve a new state of the art. In the former task our best\nmodel outperforms even all previously reported ensembles.'},
 {'type': 'image',
  'distance': 143.9729,
  'page_number': 4,
  'image_path': 'figures/page_04_img_1.png',
  'caption

In [22]:
answer_query("Explaine me attension?",top_k_text=2,top_k_img=1,top_k_overall=3)


=== Query: Explaine me attension? ===

Rank 1 | type: text | distance: 44.8092
page: 9
text: 2 6.11 23.7 36 4 5.19 25.3 50 8 4.88 25.5 80 256 32 32 5.75 24.5 28 1024 128 128 4.66 26.0 168 1024 5.12 25.4 53 4096 4.75 26.2 90

Rank 2 | type: text | distance: 45.1418
page: 3
text: We call our particular attention "Scaled Dot-Product Attention" (Figure 2). The input consists of queries and keys of dimension dk, and values of dimension dv. We compute the dot products of the

Rank 3 | type: image | distance: 207.2857
page: 4
image path: figures/page_04_img_1.png
caption: Figure 2: (left) Scaled Dot-Product Attention. (right) Multi-Head Attention consists of several
attention layers running in parallel.


[{'type': 'text',
  'distance': 44.80922,
  'page_number': 9,
  'content': '2\n6.11\n23.7\n36\n4\n5.19\n25.3\n50\n8\n4.88\n25.5\n80\n256\n32\n32\n5.75\n24.5\n28\n1024\n128\n128\n4.66\n26.0\n168\n1024\n5.12\n25.4\n53\n4096\n4.75\n26.2\n90'},
 {'type': 'text',
  'distance': 45.141823,
  'page_number': 3,
  'content': 'We call our particular attention "Scaled Dot-Product Attention" (Figure 2). The input consists of\nqueries and keys of dimension dk, and values of dimension dv. We compute the dot products of the'},
 {'type': 'image',
  'distance': 207.2857,
  'page_number': 4,
  'image_path': 'figures/page_04_img_1.png',
  'caption': 'Figure 2: (left) Scaled Dot-Product Attention. (right) Multi-Head Attention consists of several\nattention layers running in parallel.',
  'content': 'Figure 2: (left) Scaled Dot-Product Attention. (right) Multi-Head Attention consists of several\nattention layers running in parallel.'}]

In [23]:
answer_query("explaine Transformer model architecture?",top_k_text=2,top_k_img=1,top_k_overall=3)


=== Query: explaine Transformer model architecture? ===

Rank 1 | type: text | distance: 24.3847
page: 2
text: To the best of our knowledge, however, the Transformer is the ﬁrst transduction model relying entirely on self-attention to compute representations of its input and output without using sequence- aligned RNNs or convolution. In the following sections, we will ...

Rank 2 | type: text | distance: 32.1493
page: 9
text: For translation tasks, the Transformer can be trained signiﬁcantly faster than architectures based on recurrent or convolutional layers. On both WMT 2014 English-to-German and WMT 2014 English-to-French translation tasks, we achieve a new state of the art. In ...

Rank 3 | type: image | distance: 147.6756
page: 4
image path: figures/page_04_img_1.png
caption: Figure 2: (left) Scaled Dot-Product Attention. (right) Multi-Head Attention consists of several
attention layers running in parallel.


[{'type': 'text',
  'distance': 24.384739,
  'page_number': 2,
  'content': 'To the best of our knowledge, however, the Transformer is the ﬁrst transduction model relying\nentirely on self-attention to compute representations of its input and output without using sequence-\naligned RNNs or convolution. In the following sections, we will describe the Transformer, motivate\nself-attention and discuss its advantages over models such as [14, 15] and [8].'},
 {'type': 'text',
  'distance': 32.14933,
  'page_number': 9,
  'content': 'For translation tasks, the Transformer can be trained signiﬁcantly faster than architectures based\non recurrent or convolutional layers. On both WMT 2014 English-to-German and WMT 2014\nEnglish-to-French translation tasks, we achieve a new state of the art. In the former task our best\nmodel outperforms even all previously reported ensembles.'},
 {'type': 'image',
  'distance': 147.67557,
  'page_number': 4,
  'image_path': 'figures/page_04_img_1.png',
  'capti